In [33]:
#v2 kinguin scraper

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

############
# get text and game name
# $future task: enable chrome headless mode, per https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug/49582462#49582462
############
print('What game are you searching for? ')
game_title = input().lower() # try 'grand theft auto' or 'skyrim'
print(game_title)
print()
url_game_title = game_title.replace(' ', '%20').replace('\'', '%27').title()
url = 'https://www.kinguin.net/listing?phrase=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('add to cart')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if 'xbox one / xbox series x|s / pc' in i:
        xbox_windows_match = i.replace('xbox one / xbox series x|s / pc', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'pc / xbox one / xbox series x|s' in i:
        xbox_windows_match = i.replace('pc / xbox one / xbox series x|s', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / series x|s / windows 10' in i:
        xbox_series_windows_match = i.replace('xbox one / series x|s / windows 10', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_series_windows_match)
    elif 'xbox one / xbox series x|s' in i:
        xbox_match = i.replace('xbox one / xbox series x|s', '`xbox one / xbox series x|s`')
        platform_clean.append(xbox_match)
    elif 'xbox one/pc' in i:
        xbox_windows_match = i.replace('xbox one/pc', '`xbox one / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / pc' in i:
        xbox_windows_match = i.replace('xbox one / pc', '`xbox one/ pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / windows 10' in i:
        xbox_windows_match = i.replace('xbox one / windows 10', '`xbox one / pc`')
        platform_clean.append(xbox_windows_match)

    elif 'xbox one' in i:
        xbox_match = i.replace('xbox one', '`xbox one`')
        platform_clean.append(xbox_match)
    elif 'xbox 360' in i:
        xbox_match = i.replace('xbox 360', '`xbox 360`')
        platform_clean.append(xbox_match)
    
    elif 'steam' in i:
        steam_match = i.replace('steam', '`steam`')
        platform_clean.append(steam_match)
        
    elif 'gog' in i:
        gog_match = i.replace('gog', '`gog`')
        platform_clean.append(gog_match)

    elif 'ps4' in i:
        playstation_match = i.replace('ps4', '`playstation 4`')
        platform_clean.append(playstation_match)
    elif 'ps5' in i:
        playstation_match = i.replace('ps5', '`playstation 5`')
        platform_clean.append(playstation_match)
    elif 'playstation 5' in i:
        playstation_match = i.replace('playstation 5', '`playstation 5`')
        platform_clean.append(playstation_match)
    
    elif 'nintendo 3ds' in i:
        nintendo_match = i.replace('nintendo 3ds', '`nintendo 3ds`')
        platform_clean.append(nintendo_match)
    elif 'nintendo switch' in i:
        nintendo_match = i.replace('nintendo switch', '`nintendo switch`')
        platform_clean.append(nintendo_match)

    else:
        platform_clean.append(i)

df = pd.DataFrame(platform_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1.columns = ['product', 'platform', 'product type', 'price'] # add column titles
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['product'] = df1['product'].str.replace('-', '') # remove - from 'region'
df1['platform'] = df1['platform'].str.replace('gog', 'gog.com')
df1['product type'] = df1['product type'].str.replace('\d+', '', regex=True) # remove digits from 'product type'
df1['product type'] = df1['product type'].str.replace(r'\.+$', '', regex=True) # remove trailing periods from 'product type'
df1['price'] = df1['price'].str.replace('$', '') # remove $ from 'prices'
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['site'] = 'Kinguin'

kinguin = df1.sort_values(by=['platform', 'price'])
kinguin.head(50)

What game are you searching for? 
skyrim



,product,platform,product type,price,site
21,anniversary upgrade dlc eu,nintendo switch,cd key,17.38,Kinguin
9,us,nintendo switch,cd key,out of stock,Kinguin
14,eu,nintendo switch,cd key,out of stock,Kinguin
4,vr eu,playstation 4,cd key,out of stock,Kinguin
11,anniversary edition eu,playstation 4,cd key,out of stock,Kinguin
17,vr us,playstation 4,cd key,out of stock,Kinguin
1,legendary edition eu,steam,cd key,10.11,Kinguin
23,special edition,steam,cd key,10.59,Kinguin
5,"legendary edition eu (without de, ch, no, rs)",steam,cd key,10.82,Kinguin
8,legendary edition row,steam,cd key,11.59,Kinguin


In [23]:
url = 'https://www.g2a.com/search?query=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('offer')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if 'platform' in i:
        platform_match = i.replace('platform', '`platform')
        platform_clean.append(platform_match)

# clean matches by type
type_clean = []
for i in platform_clean:
    if 'type' in i:
        type_match = i.replace('type', '`type')
        type_clean.append(type_match)

# clean matches by region
region_clean = []
for i in type_clean:
    if 'region' in i:
        region_match = i.replace('region', '`region')
        region_clean.append(region_match)

# clean matches by activation region
activation_region_clean = []
for i in region_clean:
    if 'can' in i:
        activation_region_match = i.replace('can', '`can')
        activation_region_clean.append(activation_region_match)

df = pd.DataFrame(activation_region_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1.columns = ['product', 'platform', 'product type', 'region', 'activation region', 'price', 'original price'] # add column titles
df1 = df1.drop(columns=['activation region'])
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['platform'] = df1['platform'].str.replace('platform: ', '').str.replace('account', '')
df1['product type'] = df1['product type'].str.replace('type:', '')
df1['region'] = df1['region'].str.replace('region:', '')
df1['price'] = df1['price'].str.replace('$ ', '')
df1['original price'] = df1['original price'].str.replace('$ ', '').str.replace(r'-.*', '', regex=True) # remove % discount
df1['site'] = 'G2A'

g2a = df1.sort_values(by=['platform', 'price'])
g2a.head(50)

,product,platform,product type,region,price,original price,site
11,- pack steam key global,steam,key,global,10.23,42.84,G2A
4,- dawnguard (pc) - steam key - global,steam,key,global,10.98,21.42,G2A
2,- legendary edition steam key global,steam,key,global,11.71,42.84,G2A
0,anniversary upgrade (pc) - steam key - global,steam,key,global,16.52,21.42,G2A
9,anniversary edition (pc) - steam key - europe,steam,key,europe,22.57,53.56,G2A
10,hearthfire (pc) - steam key - global,steam,key,global,24.68,None,G2A
5,vr (pc) - steam key - europe,steam,key,europe,25.30,64.27,G2A
3,- dragonborn (pc) - steam key - global,steam,key,global,7.97,21.42,G2A
1,(pc) - steam key - global,steam,key,global,8.57,42.84,G2A
13,special edition + fallout 4 g.o.t.y bundle ste...,steam,key,global,out of stock,None,G2A


In [24]:
url = 'https://www.cdkeys.com/?q=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('add to wish list')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if '(' in i:
        platform_match = i.replace('(', '`(')
        platform_clean.append(platform_match)

# clean matches by platform, pt2
platform2_clean = []
for i in platform_clean:
    if 'xbox one & xbox series x|s' in i:
        xbox_windows_match = i.replace('xbox one & xbox series x|s', 'xbox one / series x|s')
        platform2_clean.append(xbox_windows_match)

    else:
        platform2_clean.append(i)

# clean matches by region
region_clean = []
for i in platform2_clean:
    if ')' in i:
        region_match = i.replace(')', ')`')
        region_clean.append(region_match)

# clean matches by type
type_clean = []
for i in region_clean:
    if 'xbox' in i:
        type_match = i.replace('xbox', '`xbox')
        type_clean.append(type_match)

df = pd.DataFrame(type_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1 = df1.drop(columns=[3])
df1.columns = ['product', 'platform', 'region', 'price', 'original price'] # add column titles
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['product'] = df1['product'].str.replace(': ', '')
df1['region'] = df1['region'].str.replace('(', '').str.replace(')', '').str.replace(')      rating:   %', '')
df1['price'] = df1['price'].str.replace('$', '')
df1['original price'] = df1['original price'].str.replace('$', '')
df1['site'] = 'CDKeys'

print('CDKeys\' <<' + game_title + '>>')
cdkeys = df1.sort_values(by=['platform', 'price'])
display(cdkeys)

CDKeys' <<skyrim>>


,product,platform,region,price,original price,site
5,anniversary edition + fallout 4 g.o.t.y bundle,xbox,eu & uk,35.59,95.19,CDKeys
1,anniversary upgrade,xbox,uk,9.79,19.29,CDKeys
8,special edition,xbox one,us,12.29,49.19,CDKeys
7,special edition and fallout g.o.t.y bundle,xbox one,uk,24.59,68.89,CDKeys
4,special edition,xbox one / series x|s,europe & uk,12.19,41.69,CDKeys
6,anniversary edition,xbox one / series x|s,turkey,13.39,59.49,CDKeys
0,anniversary edition,xbox one / series x|s,uk,19.49,58.59,CDKeys
2,anniversary edition,xbox one / series x|s,eu & uk,21.99,59.49,CDKeys
3,anniversary edition,xbox one / series x|s,us,21.99,50.59,CDKeys


In [32]:
import pandas as pd
combined = pd.concat([kinguin, g2a, cdkeys])
combined = combined[['product', 'product type', 'region', 'platform', 'price', 'original price', 'site']]
combined = combined.sort_values(by=['region', 'platform', 'price'], ascending=[True, True, True])
display(combined)

,product,product type,region,platform,price,original price,site
12,special edition xbox live key argentina,key,argentina,xbox live,5.22,42.84,G2A
9,anniversary edition (pc) - steam key - europe,key,europe,steam,22.57,53.56,G2A
5,vr (pc) - steam key - europe,key,europe,steam,25.30,64.27,G2A
11,- pack steam key global,key,global,steam,10.23,42.84,G2A
4,- dawnguard (pc) - steam key - global,key,global,steam,10.98,21.42,G2A
2,- legendary edition steam key global,key,global,steam,11.71,42.84,G2A
0,anniversary upgrade (pc) - steam key - global,key,global,steam,16.52,21.42,G2A
10,hearthfire (pc) - steam key - global,key,global,steam,24.68,None,G2A
3,- dragonborn (pc) - steam key - global,key,global,steam,7.97,21.42,G2A
1,(pc) - steam key - global,key,global,steam,8.57,42.84,G2A
